# Day 14

In [1]:
import os

In [2]:
input_file_path = os.path.join(".", "day14.txt")
with open(input_file_path, 'r') as reader:
    input_data = reader.read()

## Part 1
Continuous sequence splicing

In [3]:
# Naive algorithm that tracks the actaul string
from collections import Counter

initial = None
instructions = {}

# Parse instructions
for line in input_data.split("\n"):
    if "->" in line:
        sequence, insertion = line.split(" -> ")
        instructions[sequence] = sequence[0] + insertion + sequence[1]
    elif initial is None:
        initial = line
    else:
        continue

# Process instructions
steps = 10
word = initial
for step in range(steps):
    sequences = []

    # Process each 2-gram
    for i, (a, b) in enumerate(zip(word[:-1], word[1:])):
        ch = a + b
        if ch in instructions:
            token = instructions[ch]
        
            # Account for overlaps
            if i > 0:
                token = token[1:]

        else:
            token = ch
        sequences.append(token)

    # Generate new word
    word = "".join(sequences)

counts = Counter(word).most_common()
delta = counts[0][1] - counts[-1][1]
print(f"Part 1: delta between most and least common element is {delta:,}")

Part 1: delta between most and least common element is 3,058


## Part 2
Alternative algorithm to handle long mutations

In [4]:
# Alternative algorithm to only track letter occurances, not ordering
import numpy as np
from collections import defaultdict

initial = None
graph = {}
steps = 40

# Parse instructions
for line in input_data.split("\n"):
    if "->" in line:
        sequence, insertion = line.split(" -> ")
        expansion = sequence[0] + insertion + sequence[1]
        graph[sequence] = {expansion[:2], expansion[1:]}
    elif initial is None:
        initial = line
    elif len(line) > 0:
        raise RuntimeError(f"Unknown instruction: {line}")

# An array to indicate when we have x units of one sequence,
# what sequences we should receive after one step
columns = list(graph.keys())
sequence_cols = {k: i for i, k in enumerate(columns)}
n = len(columns)
arr_step = np.zeros((n, n))

# Populate transition matrix
for parent, (left, right) in graph.items():
    x = sequence_cols[parent]
    i = sequence_cols[left]
    j = sequence_cols[right]
    arr_step[x, [i, j]] += 1

# Initial vector
arr_initial = np.zeros(n)
for x, y in zip(initial[:-1], initial[1:]):
    col = sequence_cols[x + y]
    arr_initial[col] += 1

# Apply steps
arr = arr_initial @ np.linalg.matrix_power(arr_step, steps)

# Derive counts
counter = defaultdict(int)
for i, sequence in enumerate(columns):
    x, y = sequence
    count = int(arr[i])
    counter[x] += count
    counter[y] += count

# Adjust for overlap on first and last character
counter[initial[0]] += 1
counter[initial[-1]] += 1

# Difference between the most common and least common element
counts = sorted([(v // 2, k) for k, v in counter.items()])
delta = counts[-1][0] - counts[0][0]
print(f"Part 2: delta between most and least common element is {delta:,}")

Part 2: delta between most and least common element is 3,447,389,044,530
